In [4]:
from numpy import array, hstack
from keras.models import Model
from keras.layers import Input, Dense

# Function to split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # Find the end of this pattern
        end_ix = i + n_steps
        # Check if we are beyond the dataset
        if end_ix > len(sequences) - 1:
            break
        # Gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# Define input sequences (new dataset)
in_seq1 = array([100, 200, 300, 400, 500, 600, 700, 800, 900])
in_seq2 = array([150, 250, 350, 450, 550, 650, 750, 850, 950])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])
# Reshape to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# Choose a number of time steps
n_steps = 3
# Convert into input/output
X, y = split_sequences(dataset, n_steps)
# Flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
# Separate output
y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))

# Define the model
visible = Input(shape=(n_input,))
dense = Dense(100, activation='relu')(visible)
# Define outputs
output1 = Dense(1)(dense)
output2 = Dense(1)(dense)
output3 = Dense(1)(dense)
# Tie together
model = Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer='adam', loss='mse')

# Fit model
history = model.fit(X, [y1, y2, y3], epochs=2000, verbose=0)

# Demonstrate prediction with new input
x_input = array([[700, 750, 1450], [800, 850, 1650], [900, 950, 1850]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)

# Display results in terminal
print("\nInput-Output Pairs:")
print("| Input (in_seq1, in_seq2, out_seq)                     | Output 1 (in_seq1) | Output 2 (in_seq2) | Output 3 (out_seq) |")
print("|-------------------------------------------------------|--------------------|--------------------|--------------------|")
for i in range(len(X)):
    input_values = X[i].reshape((n_steps, 3))  # Reshape back to the original input structure
    output_values = [y1[i][0], y2[i][0], y3[i][0]]
    print(f"| {input_values.tolist()}  | {output_values[0]:>18.2f} | {output_values[1]:>18.2f} | {output_values[2]:>18.2f} |")

print("\nPrediction:")
print(f"Given input sequence {x_input.flatten().tolist()}, the predicted values are:")
print(f"Output 1 (in_seq1): {yhat[0][0][0]:.2f}")
print(f"Output 2 (in_seq2): {yhat[1][0][0]:.2f}")
print(f"Output 3 (out_seq): {yhat[2][0][0]:.2f}")


Input-Output Pairs:
| Input (in_seq1, in_seq2, out_seq)                     | Output 1 (in_seq1) | Output 2 (in_seq2) | Output 3 (out_seq) |
|-------------------------------------------------------|--------------------|--------------------|--------------------|
| [[100, 150, 250], [200, 250, 450], [300, 350, 650]]  |             400.00 |             450.00 |             850.00 |
| [[200, 250, 450], [300, 350, 650], [400, 450, 850]]  |             500.00 |             550.00 |            1050.00 |
| [[300, 350, 650], [400, 450, 850], [500, 550, 1050]]  |             600.00 |             650.00 |            1250.00 |
| [[400, 450, 850], [500, 550, 1050], [600, 650, 1250]]  |             700.00 |             750.00 |            1450.00 |
| [[500, 550, 1050], [600, 650, 1250], [700, 750, 1450]]  |             800.00 |             850.00 |            1650.00 |
| [[600, 650, 1250], [700, 750, 1450], [800, 850, 1650]]  |             900.00 |             950.00 |            1850.00 |

Predict